# AI Orchestrator - RAG Basic

user query -> search -> generate answer

## Prep

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

> update the name of environment variables(`AZURE_OPENAI_KEY`, `AZURE_OPENAI_ENDPOINT`) and the value of `index_name`

In [13]:
import os
from openai import AzureOpenAI

#MODEL = "gpt-4o"
MODEL = "gpt-35-turbo"
client = AzureOpenAI(
    api_key=os.environ['AZURE_OPENAI_KEY'],
    #api_version="2024-10-01-preview",
    api_version="2024-08-01-preview",
    azure_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']
)

In [14]:
def _chat_gpt(messages, model=MODEL, temp=0, topp=0.1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp,
        max_tokens=2000,
        top_p=topp
    )   
    
    return response.choices[0].message.content

In [15]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZSCH_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

index_name = os.environ["AZSCH_INDEX_NAME"]

## Search query

In [16]:
query = "What is OpenAI assistant api?"  
chat_history = []

## Search

Vector Search
- vectorquery = "query"
- search_text = None

In [31]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

topK = 2

# Pure Vector Search
search_client = SearchClient(endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topK, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk"],
    top=topK
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"  chunk_id: {result['chunk_id']}")  
    print(f"  Score: {result['@search.score']}")  
    #print(f"Content: {result['chunk']}")

parent_id: assistants-python.md
  chunk_id: 0
  Score: 0.85328734
parent_id: assistant.md
  chunk_id: 0
  Score: 0.8505703


In [32]:
def get_context(text):
    vector_query = VectorizableTextQuery(text=text, k_nearest_neighbors=topK, fields="vector", exhaustive=True)
    results = search_client.search(  
        search_text=None,  
        vector_queries= [vector_query],
        select=["parent_id", "chunk_id", "chunk", "title"],
        top=topK
    )  

    context = ""
    metadata = []
    for result in results:  
        context = context + result['chunk'] + "\n\n"
        metadata.append({"score": result['@search.score'], "chunk_id": result['chunk_id'], "parent_id": result['parent_id'], "title": result["title"]})

    return context, metadata

In [33]:
context, metadata = get_context(query)

## Response

In [34]:
# prompt templates
from jinja2 import Template
from common import parse_chat

with open('./reply_simple.jinja2') as file:
    simple_response_template = file.read()

with open('./reply_simple_ground.jinja2') as file:
    ground_response_template = file.read()

In [35]:
prompt = Template(simple_response_template, trim_blocks=True, keep_trailing_newline=True).render(
    context=context,
    chat_history=chat_history,
    user_query=query
)

print(prompt)

system:
You are a helpful and friendly assistant. Generate answer based on the context provided below. 
If you user asked in Korean then answer back in Korean otherwise answer in English only.

## Context
<a href="https://github.com/openai/openai-python" target="_blank">Library source code</a> | <a href="https://pypi.org/project/openai/" target="_blank">Package (PyPi)</a> |

## Prerequisites

- An Azure subscription - <a href="https://azure.microsoft.com/free/cognitive-services" target="_blank">Create one for free</a>
- Access granted to Azure OpenAI in the desired Azure subscription

    Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form at <a href="https://aka.ms/oai/access" target="_blank">https://aka.ms/oai/access</a>. Open an issue on this repo to contact us if you have an issue.
- <a href="https://www.python.org/" target="_blank">Python 3.7.1 or later version</a>
- The following Python libraries: os, j

In [36]:
def get_response(query, chat_history, template, model=MODEL):

    context, metadata = get_context(query)
    
    prompt = Template(template, trim_blocks=True, keep_trailing_newline=True).render(
        context=context,
        chat_history=chat_history,
        user_query=query
    )
    
    messages = parse_chat(prompt)
    
    return _chat_gpt(messages, model), metadata

## Very Simple Prompt

In [37]:
chat_history = []
query = "What is OpenAI assistant api?"  
response, metadata = get_response(query, chat_history, simple_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is OpenAI assistant api? 
>  The OpenAI Assistants API is a feature of Azure OpenAI Service that allows developers to create applications with sophisticated copilot-like experiences. It enables developers to build custom AI assistants that can sift through data, suggest solutions, and automate tasks. The Assistants API provides a stateful evolution of the chat completion API, supporting persistent automatically managed threads and allowing developers to easily manage conversation state and chat threads. It also provides access to tools such as code interpreter and function calling. The Assistants API is built on the same capabilities as OpenAI's GPT product and can be used for various use cases such as AI-powered product recommenders, coding assistants, and chatbots.


In [38]:
metadata

[{'score': 0.879656,
  'chunk_id': '0',
  'parent_id': 'assistants.md',
  'title': 'Azure OpenAI Service Assistant API concepts'},
 {'score': 0.86666685,
  'chunk_id': '0',
  'parent_id': 'assistant.md',
  'title': 'How to create Assistants with Azure OpenAI Service'}]

In [39]:
query = "What is Google"
response, metadata = get_response(query, chat_history, simple_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is Google 
>  Google is a multinational technology company that specializes in Internet-related services and products. It was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University. Google is best known for its search engine, which is the most widely used search engine in the world. In addition to search, Google offers a wide range of products and services, including online advertising technologies, cloud computing, software, hardware, and more. Google's mission is to organize the world's information and make it universally accessible and useful.


In [40]:
metadata

[{'score': 0.7866772,
  'chunk_id': '6',
  'parent_id': 'prompt-chat-completion.md',
  'title': 'How to work with prompt engineering and the Chat Completion API'},
 {'score': 0.7863357,
  'chunk_id': '4',
  'parent_id': 'prompt-engineering.md',
  'title': 'Azure OpenAI Service | Introduction to Prompt engineering'}]

## Grounded Prompt

ground response using `ground_response_template`

In [41]:
chat_history = []
query = "What is OpenAI assistant api?"  
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is OpenAI assistant api? 
>  The OpenAI Assistants API is a feature of Azure OpenAI Service that allows developers to create applications with sophisticated copilot-like experiences. It enables developers to build custom AI assistants that can sift through data, suggest solutions, and automate tasks. The Assistants API provides a stateful solution for managing conversation state, chat threads, tool integrations, retrieval documents, and code execution. It supports persistent automatically managed threads and allows developers to access multiple tools in parallel. The API is built on the capabilities of OpenAI's GPT product and can be used for various use cases such as AI-powered product recommenders, coding assistants, employee Q&A chatbots, and more.


In [42]:
query = "What is Google"
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is Google 
>  Sorry, I don't know.


## Condensed Question

example of condense question:

- first question "what is assistant api?"
- second question "show sample code in python"

intent of second question is "show sample code for assistant api in python"

In [43]:
chat_history = []
query = "What is OpenAI assistant api?"  
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is OpenAI assistant api? 
>  The OpenAI Assistants API is a feature of Azure OpenAI Service that allows developers to create applications with sophisticated copilot-like experiences. It enables developers to build custom AI assistants that can sift through data, suggest solutions, and automate tasks. The Assistants API provides a stateful solution for managing conversation state, chat threads, tool integrations, retrieval documents, and code execution. It supports persistent automatically managed threads and allows developers to access multiple tools in parallel. The API is built on the capabilities of OpenAI's GPT product and can be used for various use cases such as AI-powered product recommenders, coding assistants, employee Q&A chatbots, and more.


> unrelated response by retrieving wrong documents

In [44]:
query = "show sample code in python"
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

show sample code in python 
>  Sure! Here's an example of a Python code snippet:

```python
# This is a simple Python program
# that prints "Hello, World!" to the console

print("Hello, World!")
```

This code will output the text "Hello, World!" when executed. Let me know if you need any further assistance!


In [45]:
query = "show OpenAI assistant api sample code in python"
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

show OpenAI assistant api sample code in python 
>  Sure! Here's an example of how you can use the OpenAI Assistant API in Python:

```python
import os
from openai import AzureOpenAI

# Create an instance of the AzureOpenAI class
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Define your assistant parameters
assistant_id = "<your-assistant-id>"
message = {
    "role": "system",
    "content": "Hello, how can I assist you today?"
}

# Call the assistant API
response = client.assistants.create_message(assistant_id, message)

# Print the assistant's response
print(response["results"][0]["content"])
```

Make sure to replace `<your-assistant-id>` with the ID of your assistant. You'll also need to set the `AZURE_OPENAI_KEY` and `AZURE_OPENAI_ENDPOINT` environment variables with your Azure OpenAI API key and endpoint respectively.

Let me know if you have any other questions!
